# VGG-16, 19 학습 및 성능 평가

## 파라미터 설정

- image size=(288,432,3)
- optimizer=rmsprop
- learning rate=2e-5
- loss=binary_crossentropy
- epoch=300
- early_stop = EarlyStopping(patience=10, monitor='val_loss')
- batchSize = 16

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/비타민 프젝1')

Mounted at /content/drive


In [2]:
# gpu 사용 가능한지 확인
# 사용 가능하다면 device_type에 CPU, GPU 2개가 뜸

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6042739245035329303
 xla_global_id: -1]

## 1. 라이브러리 임포트

In [3]:
import os
import numpy as np
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt


from tensorflow.keras import Input
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D , Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight

## 2-1. 모델 레이어 쌓기: VGG-13

In [4]:
input_tensor = Input(shape=(288,432,3), dtype='float32', name='input')
# input_tensor = Input(shape=(244,244,3), dtype='float32', name='input')

x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dense(4096, kernel_initializer='he_normal')(x)
x = Dense(1024, kernel_initializer='he_normal')(x)
x = Dense(100, kernel_initializer='he_normal')(x)

output_tensor = Dense(1, activation='sigmoid')(x)

vgg13 = Model(input_tensor, output_tensor)
vgg13.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 288, 432, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 288, 432, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 288, 432, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 144, 216, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 144, 216, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 144, 216, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 72, 108, 128)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 72, 108, 256)        │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 72, 108, 256)        │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 36, 54, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 36, 54, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 36, 54, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 18, 27, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 18, 27, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 18, 27, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 9, 13, 512)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 59904)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4096)                │     245,370,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │       4,195,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 100)                 │         102,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 259,073,801 (988.29 MB)

 Trainable params: 259,073,801 (988.29 MB)

 Non-trainable params: 0 (0.00 B)

## 2-2. 모델 레이어 쌓기: VGG-16

In [ ]:
input_tensor = Input(shape=(288,432,3), dtype='float32', name='input')
# input_tensor = Input(shape=(244,244,3), dtype='float32', name='input')

x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dense(4096, kernel_initializer='he_normal')(x)
x = Dense(1024, kernel_initializer='he_normal')(x)
x = Dense(100, kernel_initializer='he_normal')(x)

output_tensor = layers.Dense(1, activation='sigmoid')(x)

vgg16 = Model(input_tensor, output_tensor)
vgg16.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)                   │ (None, 288, 432, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 288, 432, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 288, 432, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 144, 216, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 144, 216, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 144, 216, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 72, 108, 128)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 72, 108, 256)        │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 72, 108, 256)        │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 72, 108, 256)        │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 36, 54, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 36, 54, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 36, 54, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 36, 54, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 18, 27, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_20 (Conv2D)                   │ (None, 18, 27, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 18, 27, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 18, 27, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 9, 13, 512)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 59904)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 4096)                │     245,370,880 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 264,383,497 (1008.54 MB)

 Trainable params: 264,383,497 (1008.54 MB)

 Non-trainable params: 0 (0.00 B)

## 2-3. 모델 레이어 쌓기: VGG-19

In [ ]:
input_tensor = Input(shape=(288,432,3), dtype='float32', name='input')
# input_tensor = Input(shape=(244,244,3), dtype='float32', name='input')

x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dense(4096, kernel_initializer='he_normal')(x)
x = Dense(1024, kernel_initializer='he_normal')(x)
x = Dense(100, kernel_initializer='he_normal')(x)

output_tensor = layers.Dense(1, activation='sigmoid')(x)

vgg19 = Model(input_tensor, output_tensor)
vgg19.summary()


Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 288, 432, 3)]     0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 288, 432, 64)      1792      
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 288, 432, 64)      36928     
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 144, 216, 64)      0         
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 144, 216, 128)     73856     
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 144, 216, 128)     147584    
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 72, 108, 128)    

## 3. 모델 컴파일, 학습시키기

In [ ]:
import os
import shutil
from tqdm import tqdm

# 원본 폴더 경로
folder_1 = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_fear_1'
folder_2 = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_fear_2'

# 병합될 대상 폴더 경로
merged_folder = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector/spec_fear'

# 병합 폴더 생성
os.makedirs(merged_folder, exist_ok=True)

# 폴더 병합 함수
def merge_folders(source_folder, target_folder):
    for file_name in tqdm(os.listdir(source_folder), desc=f"Processing {source_folder}"):
        source_path = os.path.join(source_folder, file_name)
        target_path = os.path.join(target_folder, file_name)
        if os.path.isfile(source_path):
            # 파일이 중복될 경우 덮어쓰기 방지
            if not os.path.exists(target_path):
                shutil.copy2(source_path, target_path)  # 메타데이터까지 복사
            else:
                print(f"Skipped duplicate file: {file_name}")

# 각 폴더 병합 실행
merge_folders(folder_1, merged_folder)
merge_folders(folder_2, merged_folder)

print(f"Files from {folder_1} and {folder_2} have been merged into {merged_folder}")

Processing /content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_fear_1: 100%|██████████| 967/967 [00:38<00:00, 25.17it/s]
Processing /content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_fear_2: 100%|██████████| 1015/1015 [00:41<00:00, 24.42it/s]

Files from /content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_fear_1 and /content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_fear_2 have been merged into /content/drive/My Drive/비타민 프젝1/Trauma-Detector/spec_fear


In [ ]:
import os
import shutil
from tqdm import tqdm

# 원본 폴더 경로
folder_1 = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_nofear_1'
folder_2 = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_nofear_2'

# 병합될 대상 폴더 경로
merged_folder = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector/spec_nofear'

# 병합 폴더 생성
os.makedirs(merged_folder, exist_ok=True)

# 폴더 병합 함수
def merge_folders(source_folder, target_folder):
    for file_name in tqdm(os.listdir(source_folder), desc=f"Processing {source_folder}"):
        source_path = os.path.join(source_folder, file_name)
        target_path = os.path.join(target_folder, file_name)
        if os.path.isfile(source_path):
            # 파일이 중복될 경우 덮어쓰기 방지
            if not os.path.exists(target_path):
                shutil.copy2(source_path, target_path)  # 메타데이터까지 복사
            else:
                print(f"Skipped duplicate file: {file_name}")

# 각 폴더 병합 실행
merge_folders(folder_1, merged_folder)
merge_folders(folder_2, merged_folder)

print(f"Files from {folder_1} and {folder_2} have been merged into {merged_folder}")

Processing /content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_nofear_1: 100%|██████████| 1000/1000 [00:49<00:00, 20.08it/s]
Processing /content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_nofear_2: 100%|██████████| 1000/1000 [00:54<00:00, 18.29it/s]

Files from /content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_nofear_1 and /content/drive/My Drive/비타민 프젝1/Trauma-Detector/processed_spec_nofear_2 have been merged into /content/drive/My Drive/비타민 프젝1/Trauma-Detector/spec_nofear


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers

# 데이터 경로 설정
base_dir = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector'
fear_dir = os.path.join(base_dir, 'spec_fear')
nofear_dir = os.path.join(base_dir, 'spec_nofear')

# 학습 및 검증 데이터 디렉토리 생성
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# 클래스별 디렉토리 생성
train_fear_dir = os.path.join(train_dir, 'fear')
train_nofear_dir = os.path.join(train_dir, 'nofear')
val_fear_dir = os.path.join(val_dir, 'fear')
val_nofear_dir = os.path.join(val_dir, 'nofear')

os.makedirs(train_fear_dir, exist_ok=True)
os.makedirs(train_nofear_dir, exist_ok=True)
os.makedirs(val_fear_dir, exist_ok=True)
os.makedirs(val_nofear_dir, exist_ok=True)

# 데이터 파일 리스트
fear_files = [os.path.join(fear_dir, f) for f in os.listdir(fear_dir) if f.endswith('.png')]
nofear_files = [os.path.join(nofear_dir, f) for f in os.listdir(nofear_dir) if f.endswith('.png')]

# 데이터셋 분리 (8:2)
fear_train, fear_val = train_test_split(fear_files, test_size=0.2, random_state=42)
nofear_train, nofear_val = train_test_split(nofear_files, test_size=0.2, random_state=42)

# 파일 이동
def copy_files(file_list, target_dir):
    for file_path in file_list:
        shutil.copy(file_path, target_dir)

# 학습 데이터 복사
copy_files(fear_train, train_fear_dir)
copy_files(nofear_train, train_nofear_dir)

# 검증 데이터 복사
copy_files(fear_val, val_fear_dir)
copy_files(nofear_val, val_nofear_dir)

In [6]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

train_dir = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector/dataset/train'
val_dir = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector/dataset/test'

class SpectrumDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_map = {}

        label_idx = 0
        for label in os.listdir(data_dir):
            label_dir = os.path.join(data_dir, label)
            if os.path.isdir(label_dir):
                if label not in self.label_map:
                    self.label_map[label] = label_idx
                    label_idx += 1
                for image_name in os.listdir(label_dir):
                    if image_name.endswith('png'):
                        self.image_paths.append(os.path.join(label_dir, image_name))
                        self.labels.append(self.label_map[label])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')  # RGB로 변환
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label


# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 크기 조정
    transforms.ToTensor(),          # 텐서 변환
])

# 데이터셋 로딩
train_dataset = SpectrumDataset(data_dir=train_dir, transform=transform)
val_dataset = SpectrumDataset(data_dir=val_dir, transform=transform)

# DataLoader 사용
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models

# VGG13 모델 정의 (VGG11 모델을 변형)
class VGG13(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG13, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 모델 인스턴스 생성
vgg13 = VGG13(num_classes=2)

# GPU가 있을 경우 GPU로 모델을 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg13.to(device)

# 모델 요약
print(vgg13)

VGG13(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [ ]:
# 모델이 GPU에서 실행되는지 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# VGG-13 모델을 `vgg13`이라는 이름으로 로드했다고 가정
vgg13.to(device)  # 모델을 GPU나 CPU로 이동

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm  # 훈련 진행 상태를 시각화

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()  # 이진 분류일 경우, CrossEntropyLoss 사용
optimizer = optim.Adam(vgg13.parameters(), lr=0.001)

# 훈련 실행
def train_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    for epoch in range(num_epochs):
        model.train()  # 모델을 훈련 모드로 설정
        running_loss = 0.0
        correct = 0
        total = 0

        # 훈련 데이터로 훈련
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # 입력과 레이블을 디바이스로 이동

            # Gradient 초기화
            optimizer.zero_grad()

            # 순전파
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 역전파
            loss.backward()

            # 최적화
            optimizer.step()

            # 손실과 정확도 계산
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Epoch 종료 후 손실과 정확도 출력
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

        # 검증 데이터로 모델 평가
        model.eval()  # 모델을 평가 모드로 설정
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():  # 평가할 때는 gradient를 계산하지 않음
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        # 검증 손실과 정확도 출력
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

In [10]:
import torch
from tqdm import tqdm

def train_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=10, save_path="best_model.pth"):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    best_val_loss = float('inf')  # 초기값을 무한대로 설정
    best_val_acc = 0.0           # 초기 정확도 값

    for epoch in range(num_epochs):
        model.train()  # 모델을 훈련 모드로 설정
        running_loss = 0.0
        correct = 0
        total = 0

        # 훈련 데이터로 훈련
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)  # 입력과 레이블을 디바이스로 이동

            # Gradient 초기화
            optimizer.zero_grad()

            # 순전파
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 역전파
            loss.backward()

            # 최적화
            optimizer.step()

            # 손실과 정확도 계산
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Epoch 종료 후 손실과 정확도 출력
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

        # 검증 데이터로 모델 평가
        model.eval()  # 모델을 평가 모드로 설정
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():  # 평가할 때는 gradient를 계산하지 않음
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        # 검증 손실과 정확도 출력
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

        # 최상의 모델 저장
        if val_loss < best_val_loss:  # 검증 손실이 최소값 갱신 시
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), save_path)  # 모델의 가중치 저장
            print(f"Best model saved with Validation Loss: {val_loss:.4f} and Accuracy: {val_acc:.2f}%")

    print("Training complete.")
    print(f"Best Validation Loss: {best_val_loss:.4f}, Best Validation Accuracy: {best_val_acc:.2f}%")


In [ ]:
train_model(train_loader, val_loader, vgg13, criterion, optimizer, num_epochs=10)

100%|██████████| 290/290 [1:20:21<00:00, 16.62s/it]


Epoch 1/10, Loss: 0.6953, Accuracy: 51.54%
Validation Loss: 0.6923, Validation Accuracy: 52.18%
Best model saved with Validation Loss: 0.6923 and Accuracy: 52.18%


100%|██████████| 290/290 [1:00:42<00:00, 12.56s/it]


Epoch 2/10, Loss: 0.6924, Accuracy: 52.20%
Validation Loss: 0.6921, Validation Accuracy: 52.18%
Best model saved with Validation Loss: 0.6921 and Accuracy: 52.18%


100%|██████████| 290/290 [1:00:43<00:00, 12.56s/it]


...

Epoch 8/10, Loss: 0.6923, Accuracy: 52.20%
Validation Loss: 0.6918, Validation Accuracy: 52.18%


100%|██████████| 290/290 [1:01:09<00:00, 12.65s/it]


Epoch 9/10, Loss: 0.6923, Accuracy: 52.20%
Validation Loss: 0.6918, Validation Accuracy: 52.18%


100%|██████████| 290/290 [1:00:50<00:00, 12.59s/it]


Epoch 10/10, Loss: 0.6923, Accuracy: 52.20%
Validation Loss: 0.6919, Validation Accuracy: 52.18%
Training complete.
Best Validation Loss: 0.6918, Best Validation Accuracy: 52.18%


In [5]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

BATCHSIZE = 32
EPOCHS = 50

train_dir = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector/train'
val_dir = '/content/drive/My Drive/비타민 프젝1/Trauma-Detector/val'


train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)  # Data Augmentation
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(288, 432),
                                                    batch_size=BATCHSIZE,
                                                    class_mode='binary',
                                                    interpolation='bilinear',
                                                    shuffle=True)

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(val_dir,
                                                    target_size=(288, 432),
                                                    batch_size=BATCHSIZE,
                                                    class_mode='binary',
                                                    interpolation='bilinear',
                                                    shuffle=False)

# 콜백 설정
early_stopping = EarlyStopping(patience=10, monitor='val_loss')
checkpoint_path = './weights/checkpoint-batch-8-epoch-{epoch:02d}-val_loss-{val_loss:.4f}.weights.h5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', verbose=1, save_weights_only=True)

# 모델 컴파일 및 학습
vgg13.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])

print("steps_per_epoch: ", len(train_generator))
print("validation_steps: ", len(valid_generator))

history = vgg13.fit(train_generator,
                              epochs=EPOCHS,
                              steps_per_epoch=len(train_generator),
                              validation_data=valid_generator,
                              validation_steps=len(valid_generator),
                              verbose=2,
                              callbacks=[early_stopping, checkpoint])

Found 3185 images belonging to 2 classes.
Found 797 images belonging to 2 classes.
steps_per_epoch:  100
validation_steps:  25
Epoch 1/50


KeyboardInterrupt: 

## 4. 모델 성능평가

### Dataset A

- evaluate A dataset

In [ ]:
BATCHSIZE = 8
EPOCHS = 300

checkpoint_path='./final_ver_weights/vgg16_rmsprop/checkpoint-batch-8-epoch-125-val_loss-0.3974.hdf5'
test_dir='./dataset/test'
test_datagen = ImageDataGenerator(rescale=1./255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(288, 432),
    batch_size=BATCHSIZE,
    class_mode='binary',
    interpolation='bilinear',
    shuffle=False)

myvgg16.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])

print("-- Evaluate --")
myvgg16.load_weights(checkpoint_path)# Re-evaluate the model
# loss, acc = myvgg16.evaluate(test_generator, steps=5)
loss, acc = myvgg16.evaluate(test_generator, steps= len(test_generator))
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


Found 2317 images belonging to 2 classes.
-- Evaluate --
290/290 [==============================] - 22s 77ms/step - loss: 0.4347 - acc: 0.9642
Restored model, accuracy: 96.42%


- predict A dataset

In [ ]:
# model = keras.models.load_model('C:\\Users\\PRLAB\\nahye_finalproject\\result_cuda_2\\dataset2-102-0.1826.hdf5')
# test_datagen = ImageDataGenerator(rescale=1./255.0)
# test_generator = test_datagen.flow_from_directory(
#         'C:\\Users\\PRLAB\\nahye_finalproject\\dataset\\test',
#         target_size=(288, 432),
#         batch_size=16,
#         class_mode='binary',
#         interpolation='bilinear',
#         shuffle=False)

checkpoint_path='./final_ver_weights/vgg16_rmsprop/checkpoint-batch-8-epoch-125-val_loss-0.3974.hdf5'
test_dir='./dataset/test'

test_datagen = ImageDataGenerator(rescale=1./255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(288, 432),
    batch_size=BATCHSIZE,
    class_mode='binary',
    interpolation='bilinear',
    shuffle=False)
myvgg16.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])



''' evaluating HTER '''
y_true = test_generator.classes
y_score = myvgg16.predict_generator(test_generator, steps = len(test_generator)).ravel()
# Calculate EER threshold
fpr, tpr, threshold = roc_curve(y_true, y_score)
fnr = 1 - tpr
eer_threshold = threshold[np.nanargmin(np.absolute((fnr-fpr)))]
# HTER
y_pred = y_score > eer_threshold
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
labels = test_generator.class_indices
print('pred_fake({})  pred_real({})\nactural_fake({})   {:12d}  {:12d}\nactual_real({})   {:12d}  {:12d}\n'.format(labels['0'], labels['1'], labels['0'], tn, fp, labels['1'], fn, tp))
hter = (fp/(tn+fp) + fn/(fn+tp)) * 0.5
# ROC curve
Accuracy = ((tn+tp) / (tn+fp+fn+tp)) * 100.0
roc_auc = roc_auc_score(y_true, y_score)
plt.figure()
lw = 2
plt.plot(fpr, tpr, 'r--', label='ROC_curve (acc_1st = %0.2f%%)' % Accuracy)
# plt.plot([0, 1], [0, 1], color = 'orange', lw=lw, linestyle='--')
plt.xlim([-0.01, 0.21])
plt.ylim([0.71, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc = "lower right")
plt.savefig('protocol_4_001.png')
plt.figure()
# print('EER: {:.4f}\tHTER: {:.4f}'.format(val_eer, hter))
# print('>> finished')

Found 2317 images belonging to 2 classes.
pred_fake(0)  pred_real(1)
actural_fake(0)           1072            36
actual_real(1)             42          1167



NameError: name 'plt' is not defined

### Dataset B

- evaluate B dataset

In [ ]:
BATCHSIZE = 8
EPOCHS = 300
checkpoint_path='C:/Users/PRLAB/final_project_hny/final_ver_weights/vgg16_rmsprop/checkpoint-batch-8-epoch-125-val_loss-0.3974.hdf5'

# test_dir2='C:/Users/PRLAB/final_project_hny/act_split_final/test'
test_dir2='C:/Users/PRLAB/nahye_finalproject/stft_spec_2'

test_datagen = ImageDataGenerator(rescale=1./255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir2,
    target_size=(288, 432),
    batch_size=BATCHSIZE,
    class_mode='binary',
    interpolation='bilinear',
    shuffle=False)

myvgg16.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])

print("-- Evaluate --")
myvgg16.load_weights(checkpoint_path)# Re-evaluate the model

# loss,acc = myvgg16.evaluate(test_generator, steps=5)
loss, acc = myvgg16.evaluate(test_generator, steps= len(test_generator))
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Found 3149 images belonging to 2 classes.
-- Evaluate --
394/394 [==============================] - 32s 81ms/step - loss: 2.3261 - acc: 0.5999
Restored model, accuracy: 59.99%


- predict B dataset

In [ ]:
# model = keras.models.load_model('C:\\Users\\PRLAB\\nahye_finalproject\\result_cuda_2\\dataset2-102-0.1826.hdf5')
# test_datagen = ImageDataGenerator(rescale=1./255.0)
# test_generator = test_datagen.flow_from_directory(
#         'C:\\Users\\PRLAB\\nahye_finalproject\\dataset\\test',
#         target_size=(288, 432),
#         batch_size=16,
#         class_mode='binary',
#         interpolation='bilinear',
#         shuffle=False)

checkpoint_path='./final_ver_weights/vgg16_rmsprop/checkpoint-batch-8-epoch-125-val_loss-0.3974.hdf5'
test_dir='C:/Users/PRLAB/nahye_finalproject/stft_spec_2'

test_datagen = ImageDataGenerator(rescale=1./255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(288, 432),
    batch_size=BATCHSIZE,
    class_mode='binary',
    interpolation='bilinear',
    shuffle=False)
myvgg16.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'])



''' evaluating HTER '''
y_true = test_generator.classes
y_score = myvgg16.predict_generator(test_generator, steps = len(test_generator)).ravel()
# Calculate EER threshold
fpr, tpr, threshold = roc_curve(y_true, y_score)
fnr = 1 - tpr
eer_threshold = threshold[np.nanargmin(np.absolute((fnr-fpr)))]
# HTER
y_pred = y_score > eer_threshold
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
labels = test_generator.class_indices
print('pred_fake({})  pred_real({})\nactural_fake({})   {:12d}  {:12d}\nactual_real({})   {:12d}  {:12d}\n'.format(labels['0'], labels['1'], labels['0'], tn, fp, labels['1'], fn, tp))
hter = (fp/(tn+fp) + fn/(fn+tp)) * 0.5
# ROC curve
Accuracy = ((tn+tp) / (tn+fp+fn+tp)) * 100.0
roc_auc = roc_auc_score(y_true, y_score)
plt.figure()
lw = 2
plt.plot(fpr, tpr, 'r--', label='ROC_curve (acc_1st = %0.2f%%)' % Accuracy)
# plt.plot([0, 1], [0, 1], color = 'orange', lw=lw, linestyle='--')
plt.xlim([-0.01, 0.21])
plt.ylim([0.71, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc = "lower right")
plt.savefig('protocol_4_001.png')
plt.figure()
# print('EER: {:.4f}\tHTER: {:.4f}'.format(val_eer, hter))
# print('>> finished')

Found 3149 images belonging to 2 classes.
pred_fake(0)  pred_real(1)
actural_fake(0)           1402           965
actual_real(1)            319           463



NameError: name 'plt' is not defined